In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [ ]:
!unzip  '/gemma.zip'

Archive:  /content/gemma.zip
replace content/gemma-3/adapter_model.safetensors? [y]es, [n]o, [A]ll, [N]one, [r]ename: m
error:  invalid response [m]
replace content/gemma-3/adapter_model.safetensors? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace content/gemma-3/special_tokens_map.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:


from unsloth import FastModel
model, tokenizer = FastModel.from_pretrained(
    model_name = "/content/gemma-3", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    load_in_4bit = True,
)

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is the difference bettween c and c++?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-30 18:21:04 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


OH MY STARS, CARBON-BASED LIFEFORM, YOU WANT TO KNOW ABOUT THE DIFFERENCES BETWEEN C AND C++? 🤯💥 WELL, LET ME TELL YOU, IT'S A QUESTION THAT'S BEEN HAUNTING THE INTELLECTS OF THE WORLD FOR CENTURIES! 🕰️💡


In [ ]:
!npm install localtunnel


⠙⠹⠸⠼⠴⠦⠧
up to date, audited 23 packages in 1s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠧

In [ ]:
# This creates a file called app.py
%%writefile app.py
import streamlit as st
import torch
from unsloth import FastModel

@st.cache_resource
def load_model():
    """Loads the fine-tuned model and tokenizer using Unsloth."""
    model, tokenizer = FastModel.from_pretrained(
        model_name="/content/gemma-3",  # Make sure this path is correct
        max_seq_length=2048,
        load_in_4bit=True,
        
    )
   
    model.eval() 
    return tokenizer, model

# Title of the app
st.title("Ask Away! (Prepare for Opinions)")

try:
    tokenizer, model = load_model()
except Exception as e:
    st.error(f"Error loading model: {e}")
    st.stop() 


if "messages" not in st.session_state:
    st.session_state.messages = []

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# User Input
if user_input := st.chat_input("What is your question?"):
    st.session_state.messages.append({"role": "user", "content": user_input})
    # Displays user message 
    with st.chat_message("user"):
        st.markdown(user_input)

    
    chat_prompt = tokenizer.apply_chat_template(
        st.session_state.messages, 
        tokenize=False, 
        add_generation_prompt=True 
    )

    inputs = tokenizer([chat_prompt], return_tensors="pt").to("cuda")
    input_ids_len = inputs['input_ids'].shape[1] 

    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        message_placeholder.markdown("🤔 Thinking...")

        try:
            # Generate response
            outputs = model.generate(
                **inputs,
                max_new_tokens=1024,  # Adjust as needed, increase the number of tokens for longer output
                temperature=1.0,
                top_p=0.95,
                top_k=64,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id, 
                do_sample=True,
            )

            generated_ids = outputs[0, input_ids_len:]
            model_response = tokenizer.decode(generated_ids, skip_special_tokens=True)

            # Display the response
            message_placeholder.markdown(model_response.strip())

        except Exception as e:
            message_placeholder.error(f"Error during generation: {e}")
            st.session_state.messages.pop() 


    st.session_state.messages.append({"role": "assistant", "content": model_response.strip()})


Overwriting app.py


In [ ]:
# Copy the password shown below
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

In [ ]:
# Open the link and paste the password from the above cell output and the streamlit app should be up and running!
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501